# Introducing Python Decorators

### Mike Driscoll



* https://github.com/driscollis/ or
* http://bit.ly/pythondecor

# The Humble Function

In [ ]:
def doubler(number):
    return number * 2

print(doubler(5))

# Functions are Objects Too

* Functions are known as "first class objects" in Python. 
* You can pass a function to another function. 
* Everything in Python is an object

In [ ]:
def doubler(number):
    return number * 2

print(doubler)
print(type(doubler))
print(doubler(10))
print(doubler.__name__)
print(doubler.__doc__)

If the function doesn't have a docstring, then `__doc__` returns `None`. Let's fix that:

In [ ]:
def doubler(number):
    """Doubles the number passed to it"""
    return number * 2 
    
print(doubler.__doc__)

# You can Assign a Functions to Variables

In [ ]:
foo = doubler
print(foo)

# Return a Function from Another Function

In [ ]:
def doubler(x):
    return x * 2

def get_doubler():
    return doubler

print(get_doubler)
d = get_doubler()
print(d(4))

# Calling a Function

In [ ]:
# To invoke a function, you just use parantheses
doubler(5)

# Determine if you can call a function via introspection

In [ ]:
hasattr(doubler, '__call__')

# Our First Decorator

**Definition:** *A decorator will take the function they are decorating and extend its behavior while not actually modifying what the function itself does.*

In [ ]:
def doubler(number):
    """Doubles the number passed to it"""
    return number * 2

def info(func):
    def wrapper(*args):
        print('Function name: ' + func.__name__)
        print('Function docstring: ' + str(func.__doc__))
        return func(*args)
    return wrapper

my_decorator = info(doubler)
print(my_decorator(2))

# Using Decorator Syntax

In [ ]:
@info
def doubler(number):
    """Doubles the number passed to it"""
    return number * 2

print(doubler(4))

# Stacking / Chaining Decorators

You can also apply multiple decorators to a single function

In [ ]:
def bold(func):
    def wrapper():
        return "<b>" + func() + "</b>"
    return wrapper

def italic(func):
    def wrapper():
        return "<i>" + func() + "</i>"
    return wrapper

@bold
@italic
def formatted_text():
    return 'Python rocks!'

print(formatted_text())

# Adding Arguments to Decorated Functions

Let's say we want to pass an argument to our `formatted_text` function to make it actually format text that is passed to it. How will that effect the decorator? Or will it?

In [ ]:
@bold
def formatted_text(text):
    return text

print(formatted_text('Python Rocks!'))

The `wrapper` function needs an argument passed to it. Let's try modifying that function and see what happens:

In [ ]:
def bold(func):
    def wrapper(*args):
        return "<b>" + func() + "</b>"
    return wrapper

@bold
def formatted_text(text):
    return text

print(formatted_text('Python Rocks!'))

Obviously we forgot something. But what?

In [ ]:
def bold(func):
    # We need *args passed to wrapper()
    def wrapper(*args):                      
        # We need to add *args here too
        return "<b>" + func(*args) + "</b>"  
    return wrapper

@bold
def formatted_text(text):
    return text

print(formatted_text('Python Rocks!'))

Now we have a fully functional decorator that allows us to pass arguments to the decorated function

## Exercise(s)

1) How do we add arguments to the decorator?

# Create a decorator with arguments

In [ ]:
def my_decorator(arg1, arg2):
    print('Decorator arg1 = ' + str(arg1))
    print('Decorator arg2 = ' + str(arg2))

    def the_real_decorator(function):

        def wrapper(*args, **kwargs):
            print('Function {} args: {} kwargs: {}'.format(
                function.__name__, str(args), str(kwargs)))
            return function(*args, **kwargs)

        return wrapper

    return the_real_decorator

# Using the arguments

In [ ]:
@my_decorator(3, 'Python')
def doubler(number):
    return number * 2

print(doubler(5))

# Unwrapping the decorator

In [ ]:
def info(arg1, arg2):
    print('Decorator arg1 = ' + str(arg1))
    print('Decorator arg2 = ' + str(arg2))
 
    def the_real_decorator(function):
 
        def wrapper(*args, **kwargs):
            print('Function {} args: {} kwargs: {}'.format(
                function.__name__, str(args), str(kwargs)))
            return function(*args, **kwargs)
 
        return wrapper
 
    return the_real_decorator

In [ ]:
def doubler(number):
    return number * 2
 
decorator = info(3, 'Python')(doubler)
print(decorator(5))

# Simplification with partial

You can simplify this example using the **functools** library

In [ ]:
from functools import partial
 
 
def info(func, arg1, arg2):
    print('Decorator arg1 = ' + str(arg1))
    print('Decorator arg2 = ' + str(arg2))
 
    def wrapper(*args, **kwargs):
        print('Function {} args: {} kwargs: {}'.format(
            func.__name__, str(args), str(kwargs)))
        return func(*args, **kwargs)
 
    return wrapper

In [ ]:
decorator_with_arguments = partial(info, arg1=3, arg2='Py')
 
@decorator_with_arguments
def doubler(number):
    return number * 2
 
print(doubler(5))

# Decorator Obfuscation

Decorators actually obfuscate some things about the original function that they are decorating. For example, the name of the function and the docstring can change:

In [ ]:
def bold(func):
    def wrapper(*args):
        return "<b>" + func(*args) + "</b>"
    return wrapper

@bold
def formatted_text(text):
    """
    Format the passed in text
    """
    return text

print(formatted_text.__name__)
print(formatted_text.__doc__)

Python provides a way to fix this in its `functools` module:

In [ ]:
from functools import wraps

def bold(func):
    @wraps(func)
    def wrapper(*args):
        return "<b>" + func(*args) + "</b>"
    return wrapper

In [ ]:
@bold
def formatted_text(text):
    """
    Format the passed in text
    """
    return text

print(formatted_text.__name__)
print(formatted_text.__doc__)

# Class Decorators

Using a class as a decorator

In [ ]:
class decorator_with_arguments:
 
    def __init__(self, arg1, arg2):
        print('in __init__')
        self.arg1 = arg1
        self.arg2 = arg2
        print('Decorator args: {}, {}'.format(arg1, arg2))
 
    def __call__(self, f):
        print('in __call__')
        def wrapped(*args, **kwargs):
            print('in wrapped()')
            return f(*args, **kwargs)
        return wrapped

In [ ]:
@decorator_with_arguments(3, 'Python')
def doubler(number):
    return number * 2
 
print(doubler(5))

# Decorating a Class

Adding a decorator to a class without modifying behavior

In [ ]:
class MyActualClass:
    def __init__(self):
        print('in MyActualClass __init__()')
 
    def quad(self, value):
        return value * 4
 
obj = MyActualClass()
print(obj.quad(4))

In [ ]:
def decorator(cls):
    class Wrapper(cls):
        def doubler(self, value):
            return value * 2
    return Wrapper

In [ ]:
@decorator
class MyActualClass:
    def __init__(self):
        print('in MyActualClass __init__()')
 
    def quad(self, value):
        return value * 4
 
obj = MyActualClass()
print(obj.quad(4))
print(obj.doubler(5))

Or just use a subclass

In [ ]:

class MyActualClass:
    def __init__(self):
        print('in MyActualClass __init__()')

    def quad(self, value):
        return value * 4

In [ ]:
class MySubClass(MyActualClass):
    def doubler(self, value):
        return value * 2

obj = MySubClass()
print(obj.quad(4))
print(obj.doubler(5))

# Practical Decorators

You can use decorators for many things. Some of the most popular uses including creating decorators for authentication (django / flask) and logging.

## A Logging Decorator

In [ ]:
import logging

def log(func):
    """
    Log what function is called
    """
    def wrap_log(*args, **kwargs):
        name = func.__name__
        logger = logging.getLogger(name)
        logger.setLevel(logging.INFO)

        # add file handler
        fh = logging.FileHandler("/home/mdriscoll/Desktop/%s.log" % name)
        fmt = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
        formatter = logging.Formatter(fmt)
        fh.setFormatter(formatter)
        logger.addHandler(fh)

        logger.info("Running function: %s" % name)
        result = func(*args, **kwargs)
        logger.info("Result: %s" % result)
        return func
    return wrap_log


@log
def doubler(number):
    """Doubles the number passed to it"""
    return number * 2


## Exercise(s)

1) How can we tell our logger decorator where to log? 
2) Can you figure out how to make the logger decorator log to stdout?


## An Example from Flask

http://flask.pocoo.org/docs/0.12/quickstart/#a-minimal-application

In [ ]:
from flask import Flask
app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'

# Questions?

